In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/youtube-comments-datas/gameTrailer_FarCry6.csv
/kaggle/input/youtube-comments-datas/apology_so_sorry.csv
/kaggle/input/youtube-comments-datas/gameTrailer_CoD.csv
/kaggle/input/youtube-comments-datas/apology_my_overdue_apology.csv
/kaggle/input/youtube-comments-datas/apology_my_response.csv
/kaggle/input/youtube-comments-datas/musicVideo_GangnamStyle.csv
/kaggle/input/youtube-comments-datas/musicVideo_Baby.csv
/kaggle/input/youtube-comments-datas/musicVideo_Friday.csv
/kaggle/input/youtube-comments-datas/gameTrailer_Pokemon.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
/kaggle/input/youtube-comments/gameTrailer_CoD.csv
/kaggle/input/youtube-comment-result/gameTrailer_result.csv


In [2]:
import zipfile
unzip = zipfile.ZipFile('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
unzip.extractall()
unzip = zipfile.ZipFile('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
unzip.extractall()

In [3]:
test_df = pd.read_csv('/kaggle/working/test.csv')
test_df.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [4]:
train_df = pd.read_csv('/kaggle/working/train.csv')
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
train_df.shape

(159571, 8)

In [6]:
train_df.loc[3,'comment_text']

'"\nMore\nI can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It\'s listed in the relevant form eg Wikipedia:Good_article_nominations#Transport  "'

In [7]:
train_df.loc[:,'toxic':].mean(axis = 0)

toxic            0.095844
severe_toxic     0.009996
obscene          0.052948
threat           0.002996
insult           0.049364
identity_hate    0.008805
dtype: float64

In [8]:
train_df[(train_df['toxic']==0) & (train_df['severe_toxic']==0) & (train_df['obscene']==0) & (train_df['threat']==0) & 
          (train_df['insult']==0) & (train_df['identity_hate']==0)].count()

id               143346
comment_text     143346
toxic            143346
severe_toxic     143346
obscene          143346
threat           143346
insult           143346
identity_hate    143346
dtype: int64

In [9]:
APPO = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"
}

In [10]:
import re

In [11]:
def clean_text(text):
    
    text = text.lower()
    text = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', '', text) # clean url
    text = re.sub(r'#(\w+)', '', text)   # clean hashes
    text = re.sub(r'@(\w+)', '', text)   # clean @
    text = re.sub(r'<[^>]+>', '', text)  # clean tags
    text = re.sub(r'\d+', '', text)      # clean digits
    text = re.sub(r'[,!@\'\"?\.$%_&#*+-:;]', '', text)   # clean punctuation
    text = [APPO[word] if word in APPO else word for word in text.split()]  #
    
    return text

In [12]:
train_df['comment_text'] = train_df['comment_text'].apply(clean_text)
test_df['comment_text'] = test_df['comment_text'].apply(clean_text)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train_set, val_set = train_test_split(train_df, test_size = 0.2, random_state=11)

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [16]:
tokenizer = Tokenizer(num_words = 100000, oov_token='<oov>')
tokenizer.fit_on_texts(train_df.comment_text)

In [17]:
traning_sequences = tokenizer.texts_to_sequences(train_set.comment_text)

In [18]:
maxlen = max([len(x) for x in np.array(traning_sequences)])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
training_padded = pad_sequences(traning_sequences, maxlen = maxlen,
                                padding = 'pre',
                                truncating='pre')
train_y = np.array(train_set.loc[:,'toxic':])

In [21]:
traning_sequences = tokenizer.texts_to_sequences(val_set.comment_text)
val_padded = pad_sequences(traning_sequences,maxlen = 1403,
                                padding = 'pre',
                                truncating='pre')
val_y = np.array(val_set.loc[:,'toxic':])

In [22]:
traning_sequences = tokenizer.texts_to_sequences(train_df.comment_text)
training_padded = pad_sequences(traning_sequences, maxlen = maxlen,
                                padding = 'pre',
                                truncating='pre')
train_y = np.array(train_df.loc[:,'toxic':])

In [23]:
testing_sequences = tokenizer.texts_to_sequences(test_df.comment_text)
test_padded = pad_sequences(testing_sequences, maxlen = maxlen,
                                padding = 'pre',
                                truncating='pre')




In [24]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(150000, 300),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences = True)),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(6, activation='sigmoid')])
                             
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['AUC'])
model.fit(x = training_padded, y = train_y, epochs = 2, batch_size = 200)

Epoch 1/2
798/798 [==============================] - 890s 1s/step - loss: 0.1375 - auc: 0.8528
Epoch 2/2
798/798 [==============================] - 884s 1s/step - loss: 0.0452 - auc: 0.9836


In [25]:
file_names = ['../input/youtube-comments-datas/gameTrailer_CoD.csv','../input/youtube-comments-datas/gameTrailer_FarCry6.csv', '../input/youtube-comments-datas/gameTrailer_Pokemon.csv'
        ,'../input/youtube-comments-datas/musicVideo_Baby.csv','../input/youtube-comments-datas/musicVideo_Friday.csv','../input/youtube-comments-datas/musicVideo_GangnamStyle.csv',
        '../input/youtube-comments-datas/apology_my_overdue_apology.csv','../input/youtube-comments-datas/apology_my_response.csv','../input/youtube-comments-datas/apology_so_sorry.csv']
file_result_name = ['gameTrailer_CoD','FarCry6','gameTrailer_Pokemon', 
                    'musicVideo_Baby', 'musicVideo_Friday','musicVideo_Gangnam', 
 'apology_my_overdue_apology', 'apology_my_response', 'apology_so_sorry'                
                   ]
for idx, file in enumerate(file_names):

    dataset = pd.read_csv(file, usecols = [0])
    dataset_save = pd.read_csv(file, usecols = [0])
    dataset.iloc[:, 0] = dataset.iloc[:, 0].apply(clean_text)
    dataset_sequences = tokenizer.texts_to_sequences(dataset.iloc[:, 0])
    dataset_padded = pad_sequences(dataset_sequences, maxlen = maxlen,
                                padding = 'pre',
                                truncating='pre')
    predicted = model.predict(dataset_padded, batch_size = 200)
    predicted_result = pd.DataFrame(predicted, columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])
    
    print("----------dataset_Save------------")
    
    
    joined = dataset_save.join(predicted_result)
    print("----------joined------------")
    print(joined.head(5))
    
    joined.columns = ['comments', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    joined.to_csv('result_{}.csv'.format(file_result_name[idx]), index = False)
    

----------dataset_Save------------
----------joined------------
  this video has 3 million dislikes. Remember that you can't escape this,black sheep of cod  \
0                             the best campaing ever                                          
1  open vid in incognito if you cant see the disl...                                          
2  Here to see the dislikes before they take them...                                          
3                  RIP DISLIKE 2005 & 2021 😂😂😂🤣🤣🤣😄😄😄                                          
4                            Where the dislikes went                                          

      toxic  severe_toxic   obscene        threat    insult  identity_hate  
0  0.016114  2.077788e-06  0.001389  5.486824e-05  0.000981       0.000223  
1  0.000682  1.236309e-09  0.000026  1.711194e-07  0.000014       0.000002  
2  0.602234  3.315710e-03  0.063833  9.468247e-03  0.199946       0.024787  
3  0.005367  1.869613e-07  0.000392  8.510292e-06  0.0002